In [121]:
# 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')

# !pip install xgboost
# !pip install tabulate
# !pip install lightgbm
# Classifier Libraries
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import collections
from sklearn.metrics import make_scorer, precision_score, recall_score, confusion_matrix, classification_report, matthews_corrcoef, cohen_kappa_score, accuracy_score, average_precision_score, roc_auc_score, precision_recall_fscore_support
from sklearn import metrics
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import  f_classif
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier

# VotingClassifier
from sklearn.ensemble import VotingClassifier

# 한글 글씨체 설정
mpl.rc('font', family = 'Malgun Gothic')
# 음수 데이터 표현
mpl.rc('axes', unicode_minus = True)
# 최대로 보일 수 있는 컬럼 설정
pd.set_option('display.max_columns', 118)

In [122]:
# 분석용 데이터 생성
df_raw = pd.read_csv("TRAIN.csv", index_col = 'actor_account') # 학습용
df_test = pd.read_csv('TEST.csv', index_col = 'actor_account') # 제출용
df_raw.drop(columns = 'Unnamed: 0', axis = 1, inplace = True)
print(df_raw.shape, df_test.shape)
df_raw.head()

(5400, 174) (3609, 172)


,103,104,110,112,113,114,115,116,118,119,125,126,127,128,129,130,137,138,139,140,141,142,143,145,146,147,148,151,154,155,156,157,158,159,play_time,161,168,169,170,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,198,...,605,606,607,608,609,610,611,612,705,719,720,722,723,log_sum,log_count,log_ip_ratio,per_day,log_ip_perday,quiz_per_playtime,quiz_per_logcount,ply_time_perlog,ply_time_perday,sit_per_time,friends_count,player_kill_count,killed_pc_pertime,killed_npc_pertime,killed_sum,port_pertime,exp_amount,exp_pertime,exp_perlog,151_pertime,151_perlog,get_AP,getap_pertime,getap_perlog,total_AP,totalap_pertime,totalap_perlog,money_get,money_use,money_get_ratio,money_use_ratio,exp_increasingrate,get_item,item_pertime,item_perlog,144,197,max_level,same_ip_rate,friend_action,assist_option,skill_del_or_get,stigmaskill_del_or_get,quest_log_count,about_region,class
actor_account,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1000127,10,10,0,0,5,0,0,0,78,78,7,2,0,2,0,0,27,5,32,1,4,70,762,16,0,0,3,0,99,99,191,1,0,1,212,46,19,2,12,0,0,1,1,0,1,0,3,35,207,111,3,3,96,96,1,0,0,0.0,0,...,0,0,0.0,0,0,0.0,0,0.0,5,224,0,0,0,5215,6.0,6.0,4,10.0,0.0000,0.0000,62.2667,212.0,0.6386,0.0,4.0,1.1032,0.4322,1.5354,1.7399,676474.0,23641.7149,29249.5614,0.0,0.0,173799.0,9483.0794,171018.2000,173799.0,9483.0794,171018.2000,6.1688,2.2269,106.8667,29.5333,-100713.1206,863.0,8.7885,207.6667,0.0,0.0,50.0,1.6667,1,0,73,40,216,0.0,0
1000344,18,17,0,0,1,0,0,0,43,42,6,14,2,12,0,0,24,19,49,0,0,111,1420,36,2,0,18,0,139,139,533,0,0,0,313,70,43,0,18,0,0,53,53,0,9,0,1,138,935,126,13,12,125,124,0,0,0,0.0,0,...,0,0,0.0,0,0,0.0,0,0.0,2,585,0,0,0,8144,11.0,6.0,4,18.0,0.0000,0.0000,81.8167,313.0,0.4874,0.0,2.0,0.2621,0.2327,0.4948,1.3453,41062625.0,23004.0585,547273.9751,0.0,0.0,50801.0,1061.6971,25891.7333,50801.0,1061.6971,25891.7333,13.8850,1.4405,308.5667,28.1333,687725.4463,1042.0,15.1698,298.9500,0.0,0.0,50.0,1.6364,3,0,26,102,396,0.0,0
1000779,11,12,0,0,1,0,0,0,84,83,0,0,0,0,0,0,21,18,42,0,0,61,5585,37,0,0,0,0,0,0,0,0,0,0,964,153,0,0,17,0,0,0,0,0,0,0,0,0,9,54,0,0,1107,1107,0,0,0,0.0,0,...,0,0,0.0,0,0,0.0,0,0.0,0,43,0,0,0,66342,4.0,11.0,4,11.0,0.0000,0.0000,489.5000,964.0,0.3494,0.0,0.0,0.0974,0.0689,0.1663,0.2587,17640790.0,300.1102,17773.8447,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.0000,0.0367,0.2340,4.9166,26.3333,69767.0012,22531.0,91.1750,13480.5000,0.0,0.0,35.0,2.7500,0,0,0,0,0,0.0,1
1000949,8,9,0,0,0,0,0,1267,15,12,0,0,0,0,0,0,6,0,6,0,0,20,1,4,0,0,0,0,1640,1639,1,0,0,0,982,153,0,0,4,0,0,0,0,0,0,0,0,0,0,1,0,0,10,10,0,0,0,0.0,0,...,0,0,0.0,0,0,0.0,0,0.0,0,25,0,0,0,44151,4.0,8.0,4,8.0,5.0741,0.1138,746.0000,982.0,0.0571,0.0,0.0,0.0256,0.0000,0.0256,0.0928,468.0,0.0057,1.6364,0.0,0.0,13.0,0.0455,13.0000,13.0,0.0455,13.0000,0.0000,0.0035,0.0000,0.5000,1.6364,12201.0,49.1657,9286.5000,0.0,0.0,30.0,2.0000,0,0,0,0,0,0.0,1
1001067,22,20,0,1,9,0,0,0,4657,4643,0,0,0,0,0,0,141,51,193,0,0,186,12101,154,0,0,4,0,0,0,0,0,0,0,1861,296,2,0,116,0,0,0,0,0,0,0,0,0,1591,93,0,0,631,631,1,0,0,0.0,0,...,0,0,0.0,0,0,0.0,0,0.0,0,162,0,0,0,49255,15.0,13.5,10,22.0,0.0000,0.0000,1120.9000,1861.0,20.2832,0.0,0.0,0.5645,0.2066,0.7711,0.7527,54857978.0,2276.6131,94174.2098,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.0000,7.4506,0.3805,971.7667,62.9667,280159.7287,13408.0,56.9362,8379.2667,0.0,0.0,43.0,1.4667,0,0,142,60,940,0.0,0


In [123]:
lgbc=LGBMClassifier(n_estimators=500, learning_rate=0.05, num_leaves=32, colsample_bytree=0.2,
            reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)

embeded_lgb_selector = SelectFromModel(lgbc, max_features=60)
embeded_lgb_selector.fit(df_raw[df_raw.columns.difference(['class'])], df_raw['class'])

embeded_lgb_support = embeded_lgb_selector.get_support()
embeded_lgb_feature = df_raw[df_raw.columns.difference(['class'])].loc[:,embeded_lgb_support].columns.tolist()
print(str(len(embeded_lgb_feature)), 'selected features')

35 selected features


In [124]:
# human의 이상치 찾기
def find_max_outlier_human(d_cp, column):
    # class 0 만
    fraud_column_data = d_cp[d_cp['class']==0][column]
    
    # Q1, Q3
    q_25 = np.percentile(fraud_column_data.values, 25)
    q_75 = np.percentile(fraud_column_data.values, 75)
    
    # 이상치 최대값 계산
    iqr = q_75 - q_25
    iqr = iqr * 1.5
    highest = q_75 + iqr
    outlier_index = fraud_column_data[(fraud_column_data > highest)].index
    
    print(column, '의 이상치 최대값:', highest, '이상치 최대값 초과하는 값들 개수: ', len(outlier_index))
    
    # 이상치 최대값 반환
    return highest

# 데이터프레임 하나를 가지고 모든 컬럼을 돌면서 컬럼 내 이상치 최대값 찾기
def replace_max_outlier_human(dataframe):
    for col in dataframe.columns:
        # 한 컬럼마다 최대 이상치 값 찾기 위한 함수 호출
        highest = int(find_max_outlier_human(dataframe, str(col)))
        
        # 최대 이상치가 0이면 바꾸지 않는다
        if highest > 0:
            dataframe[str(col)][dataframe[str(col)] > highest] = highest
            
    return dataframe

In [125]:
# human 이상치 처리
df_raw = replace_max_outlier_human(df_raw)

103 의 이상치 최대값: 95.5 이상치 최대값 초과하는 값들 개수:  474
104 의 이상치 최대값: 95.5 이상치 최대값 초과하는 값들 개수:  473
110 의 이상치 최대값: 0.0 이상치 최대값 초과하는 값들 개수:  825
112 의 이상치 최대값: 0.0 이상치 최대값 초과하는 값들 개수:  447
113 의 이상치 최대값: 15.0 이상치 최대값 초과하는 값들 개수:  431
114 의 이상치 최대값: 0.0 이상치 최대값 초과하는 값들 개수:  92
115 의 이상치 최대값: 0.0 이상치 최대값 초과하는 값들 개수:  739
116 의 이상치 최대값: 0.0 이상치 최대값 초과하는 값들 개수:  517
118 의 이상치 최대값: 403.0 이상치 최대값 초과하는 값들 개수:  510
119 의 이상치 최대값: 399.5 이상치 최대값 초과하는 값들 개수:  506
125 의 이상치 최대값: 45.0 이상치 최대값 초과하는 값들 개수:  629
126 의 이상치 최대값: 56.0 이상치 최대값 초과하는 값들 개수:  254
127 의 이상치 최대값: 12.5 이상치 최대값 초과하는 값들 개수:  331
128 의 이상치 최대값: 42.5 이상치 최대값 초과하는 값들 개수:  314
129 의 이상치 최대값: 0.0 이상치 최대값 초과하는 값들 개수:  924
130 의 이상치 최대값: 2.5 이상치 최대값 초과하는 값들 개수:  375
137 의 이상치 최대값: 72.5 이상치 최대값 초과하는 값들 개수:  692
138 의 이상치 최대값: 89.625 이상치 최대값 초과하는 값들 개수:  345
139 의 이상치 최대값: 198.5 이상치 최대값 초과하는 값들 개수:  470
140 의 이상치 최대값: 5.0 이상치 최대값 초과하는 값들 개수:  223
141 의 이상치 최대값: 2.5 이상치 최대값 초과하는 값들 개수:  804
142 의 이상치 최대값: 469.0 이상치 최대값 초과하는 값들 개수:  299
143 의 이상치 최대값:

In [126]:
# x_train, x_test, y_train, y_test = train_test_split(df_raw[embeded_lgb_feature], df_raw['class'], stratify = df_raw['class'], test_size = 0.3, random_state = 1)
# print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [127]:
# 분석
x_train = df_raw[embeded_lgb_feature].copy()
y_train = df_raw['class']
x_test = df_test[embeded_lgb_feature].copy()

In [128]:
print(x_train.shape, x_test.shape, y_train.shape)

(5400, 35) (3609, 35) (5400,)


In [129]:
# RandomForestClassifier
rf = RandomForestClassifier(bootstrap=False,
                            class_weight='balanced',
                            criterion='entropy',
                            min_impurity_decrease=0.001,
                            oob_score=False,
                            verbose=1, random_state = 0)
rf.fit(x_train, y_train)
y_pred_rf = rf.predict(x_test)
# y_pred_rf_f1 = f1_score(y_pred_rf, y_test)

# BaggingClassifier
bg = BaggingClassifier(n_estimators=10, random_state=0)
bg.fit(x_train, y_train)
y_pred_bg = rf.predict(x_test)
# y_pred_bg_f1 = f1_score(y_pred_bg, y_test)

# CatBoostClassifier
cat = CatBoostClassifier(od_type='Iter', od_wait=100, learning_rate=0.01,verbose=False, random_state=0)
cat.fit(x_train, y_train)
y_pred_cat = cat.predict(x_test)
# y_pred_cat_f1 = f1_score(y_pred_cat, y_test)

# XGB
xgb = XGBClassifier(random_state=0)
xgb.fit(x_train, y_train)
y_pred_xgb = xgb.predict(x_test)
# y_pred_xgb_f1 = f1_score(y_pred_xgb, y_test)

# LGBM
lgbm = LGBMClassifier(random_state = 42,
                      learning_rate=0.05,
                      num_iterations=1000,
                      early_stopping_round=5,
                      max_depth=10,
                      num_leaves=10,
                      boosting_type='dart',
                      objective='binary',
                      metric='binary_logloss')
lgbm.fit(x_train, y_train)
y_pred_lgbm = lgbm.predict(x_test)
# y_pred_lgbm_f1 = f1_score(y_pred_lgbm, y_test)

y_pred_rf, counts_rf = np.unique(y_pred_rf, return_counts=True)
y_pred_bg, counts_bg = np.unique(y_pred_bg, return_counts=True)
y_pred_cat, counts_cat = np.unique(y_pred_cat, return_counts=True)
y_pred_xgb, counts_xgb = np.unique(y_pred_xgb, return_counts=True)
y_pred_lgbm, counts_lgbm = np.unique(y_pred_lgbm, return_counts=True)

print('rf 결과 \t====>\t',dict(zip(y_pred_rf, counts_rf)))
print('bg 결과 \t====>\t',dict(zip(y_pred_bg, counts_bg)))
print('cat 결과\t====>\t',dict(zip(y_pred_cat, counts_cat)))
print('xgb 결과\t====>\t',dict(zip(y_pred_xgb, counts_xgb)))
print('lgbm 결과\t====>\t',dict(zip(y_pred_lgbm, counts_lgbm)))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


rf 결과 	====>	 {0: 3473, 1: 136}
bg 결과 	====>	 {0: 3473, 1: 136}
cat 결과	====>	 {0: 3479, 1: 130}
xgb 결과	====>	 {0: 3466, 1: 143}
lgbm 결과	====>	 {0: 3468, 1: 141}


In [130]:
# # RandomForestClassifier
# rf = RandomForestClassifier(bootstrap=False,
#                             class_weight='balanced',
#                             criterion='entropy',
#                             min_impurity_decrease=0.001,
#                             oob_score=False,
#                             verbose=1, random_state = 0)
# rf.fit(x_train, y_train)
# y_pred_rf = rf.predict(x_test)
# y_pred_rf_f1 = f1_score(y_pred_rf, y_test)

# # BaggingClassifier
# bg = BaggingClassifier(n_estimators=10, random_state=0)
# bg.fit(x_train, y_train)
# y_pred_bg = rf.predict(x_test)
# y_pred_bg_f1 = f1_score(y_pred_bg, y_test)

# # CatBoostClassifier
# cat = CatBoostClassifier(verbose=False)
# cat.fit(x_train, y_train)
# y_pred_cat = cat.predict(x_test)
# y_pred_cat_f1 = f1_score(y_pred_cat, y_test)

# # XGB
# xgb = XGBClassifier(random_state=0)
# xgb.fit(x_train, y_train)
# y_pred_xgb = xgb.predict(x_test)
# y_pred_xgb_f1 = f1_score(y_pred_xgb, y_test)

# # LGBM
# lgbm = LGBMClassifier(random_state = 42,
#                       learning_rate=0.05,
#                       num_iterations=1000,
#                       early_stopping_round=5,
#                       max_depth=10,
#                       num_leaves=10,
#                       boosting_type='dart',
#                       objective='binary',
#                       metric='binary_logloss')
# lgbm.fit(x_train, y_train)
# y_pred_lgbm = lgbm.predict(x_test)
# y_pred_lgbm_f1 = f1_score(y_pred_lgbm, y_test)

# print('rf\t  f1 score : {:.4f}  '.format(y_pred_rf_f1))
# print('bg\t  f1 score : {:.4f}  '.format(y_pred_bg_f1))
# print('cat\t  f1 score : {:.4f}  '.format(y_pred_cat_f1))
# print('xgb\t  f1 score : {:.4f}  '.format(y_pred_xgb_f1))
# print('lgbm\t  f1 score : {:.4f}  '.format(y_pred_lgbm_f1))

In [131]:
# df_test = df_test[embeded_lgb_feature]
# df_test.head()

In [132]:
# y_pred_rf = rf.predict(df_test)
# y_pred_bg = rf.predict(df_test)
# y_pred_cat = cat.predict(df_test)
# y_pred_xgb = xgb.predict(df_test)
# y_pred_lgbm = lgbm.predict(df_test)

In [133]:
# y_pred_rf, counts_rf = np.unique(y_pred_rf, return_counts=True)
# y_pred_bg, counts_bg = np.unique(y_pred_bg, return_counts=True)
# y_pred_cat, counts_cat = np.unique(y_pred_cat, return_counts=True)
# y_pred_xgb, counts_xgb = np.unique(y_pred_xgb, return_counts=True)
# y_pred_lgbm, counts_lgbm = np.unique(y_pred_lgbm, return_counts=True)

# print('rf 결과 \t====>\t',dict(zip(y_pred_rf, counts_rf)))
# print('bg 결과 \t====>\t',dict(zip(y_pred_bg, counts_bg)))
# print('cat 결과\t====>\t',dict(zip(y_pred_cat, counts_cat)))
# print('xgb 결과\t====>\t',dict(zip(y_pred_xgb, counts_xgb)))
# print('lgbm 결과\t====>\t',dict(zip(y_pred_lgbm, counts_lgbm)))

In [134]:
models5 = [('RandomForestClassifier', rf),('BaggingClassifier', bg),('CatBoostClassifier', cat),('XGB', xgb),('LGBM', lgbm)]

# hard vote
soft_vote  = VotingClassifier(models5, voting='hard',verbose=False)
soft_vote.fit(x_train, y_train)
models5_pred = soft_vote.predict(x_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [135]:
models5_output = pd.DataFrame({'m_account':x_test.index, 'predict':models5_pred})

In [136]:
models5_pred, counts_result = np.unique(models5_pred, return_counts=True)
print('5가지 모델에 대한 Voting 결과\t====>\t',dict(zip(models_pred, counts_result)))

5가지 모델에 대한 Voting 결과	====>	 {0: 3475, 1: 134}


In [137]:
models3 = [('RandomForestClassifier', rf),('CatBoostClassifier', cat),('LGBM', lgbm)]

# hard vote
soft_vote  = VotingClassifier(models3, voting='hard',verbose=False)
soft_vote.fit(x_train, y_train)
models3_pred = soft_vote.predict(x_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [138]:
models3_output = pd.DataFrame({'m_account':x_test.index, 'predict':models3_pred})

In [139]:
models3_pred, counts_result = np.unique(models3_pred, return_counts=True)
print('3가지 모델에 대한 Voting 결과\t====>\t',dict(zip(models3_pred, counts_result)))

3가지 모델에 대한 Voting 결과	====>	 {0: 3478, 1: 131}


In [140]:
models3_output.to_csv('데마시아_01.csv', index = False)
models5_output.to_csv('데마시아_02.csv', index = False)